# Experiment 003: Lattice-Based Construction

Implement the Zaburo-style lattice construction approach:
- Trees in alternating rows with 0° and 180° angles
- Offset pattern for interlocking
- Try multiple row/column spacing combinations
- Compare with pre-optimized baseline per-N

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
from decimal import Decimal, getcontext
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

# Set precision
getcontext().prec = 25
scale_factor = Decimal('1e15')

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def create_tree_polygon(x, y, deg):
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    vertices = [(tx * cos_a - ty * sin_a + x, tx * sin_a + ty * cos_a + y) for tx, ty in zip(TX, TY)]
    return Polygon(vertices)

def compute_bounding_side(polygons):
    if not polygons:
        return 0
    all_points = []
    for poly in polygons:
        all_points.extend(list(poly.exterior.coords))
    all_points = np.array(all_points)
    min_x, min_y = all_points.min(axis=0)
    max_x, max_y = all_points.max(axis=0)
    return max(max_x - min_x, max_y - min_y)

def compute_score_for_n(df, n):
    prefix = f"{n:03d}_"
    trees = df[df['id'].str.startswith(prefix)]
    if len(trees) != n:
        return float('inf')
    polygons = [create_tree_polygon(parse_value(row['x']), parse_value(row['y']), parse_value(row['deg'])) for _, row in trees.iterrows()]
    side = compute_bounding_side(polygons)
    return side**2 / n

def compute_total_score(df):
    total = 0
    for n in range(1, 201):
        total += compute_score_for_n(df, n)
    return total

print("Functions defined")

In [ ]:
# Zaburo-style lattice construction
class ChristmasTree:
    """Represents a single, rotatable Christmas tree."""
    def __init__(self, center_x=0, center_y=0, angle=0):
        self.center_x = float(center_x)
        self.center_y = float(center_y)
        self.angle = float(angle)
        
        # Create polygon
        self.polygon = create_tree_polygon(self.center_x, self.center_y, self.angle)

def find_best_trees_zaburo(n):
    """Zaburo-style lattice construction with alternating rows."""
    best_score, best_trees = float("inf"), None
    
    for n_even in range(1, n + 1):
        for n_odd in [n_even, n_even - 1]:
            if n_odd < 0:
                continue
            all_trees = []
            rest = n
            r = 0
            while rest > 0:
                m = min(rest, n_even if r % 2 == 0 else n_odd)
                if m <= 0:
                    break
                rest -= m
                
                angle = 0 if r % 2 == 0 else 180
                x_offset = 0 if r % 2 == 0 else 0.35  # Half tree width
                y = r // 2 * 1.0 if r % 2 == 0 else (0.8 + (r - 1) // 2 * 1.0)
                
                for i in range(m):
                    tree = ChristmasTree(center_x=0.7 * i + x_offset, center_y=y, angle=angle)
                    all_trees.append(tree)
                
                r += 1
            
            if len(all_trees) != n:
                continue
                
            # Compute bounding box
            polygons = [t.polygon for t in all_trees]
            side = compute_bounding_side(polygons)
            score = side ** 2
            
            if score < best_score:
                best_score = score
                best_trees = all_trees
    
    return best_score, best_trees

# Test for N=10
score, trees = find_best_trees_zaburo(10)
print(f"N=10: score={score:.6f}, num_trees={len(trees) if trees else 0}")

In [ ]:
# Generate lattice solutions for all N from 1 to 200
print("Generating lattice solutions for N=1 to 200...")
lattice_solutions = []

for n in tqdm(range(1, 201)):
    score, trees = find_best_trees_zaburo(n)
    lattice_solutions.append((n, score, trees))

print(f"Generated {len(lattice_solutions)} solutions")

In [ ]:
# Compute total lattice score
lattice_total = sum(score / n for n, score, _ in lattice_solutions)
print(f"Lattice construction total score: {lattice_total:.6f}")

# Load baseline for comparison
df_baseline = pd.read_csv('/home/code/external_data/saspav/santa-2025.csv')
baseline_total = compute_total_score(df_baseline)
print(f"Baseline total score: {baseline_total:.6f}")
print(f"Difference: {lattice_total - baseline_total:.6f}")

In [ ]:
# Compare per-N scores
print("\nPer-N comparison (lattice vs baseline):")
print("N\tLattice\t\tBaseline\tDiff\t\tBetter")
print("-" * 70)

improved_count = 0
for n, lattice_score, _ in lattice_solutions:
    baseline_score = compute_score_for_n(df_baseline, n)
    diff = lattice_score / n - baseline_score
    better = "LATTICE" if diff < -1e-9 else ("BASELINE" if diff > 1e-9 else "SAME")
    if diff < -1e-9:
        improved_count += 1
    if n <= 20 or n >= 190 or diff < -1e-6:
        print(f"{n}\t{lattice_score/n:.6f}\t{baseline_score:.6f}\t{diff:.6f}\t{better}")

print(f"\nLattice is better for {improved_count} out of 200 N values")

In [ ]:
# Create ensemble: pick best per-N from lattice and baseline
print("Creating ensemble (best per-N from lattice and baseline)...")

ensemble_rows = []
ensemble_total = 0

for n, lattice_score, lattice_trees in lattice_solutions:
    baseline_score = compute_score_for_n(df_baseline, n)
    lattice_score_per_n = lattice_score / n
    
    if lattice_score_per_n < baseline_score - 1e-9:
        # Use lattice solution
        for i, tree in enumerate(lattice_trees):
            ensemble_rows.append({
                'id': f"{n:03d}_{i}",
                'x': f"s{tree.center_x}",
                'y': f"s{tree.center_y}",
                'deg': f"s{tree.angle}"
            })
        ensemble_total += lattice_score_per_n
    else:
        # Use baseline solution
        prefix = f"{n:03d}_"
        baseline_trees = df_baseline[df_baseline['id'].str.startswith(prefix)]
        for _, row in baseline_trees.iterrows():
            ensemble_rows.append(row.to_dict())
        ensemble_total += baseline_score

print(f"Ensemble total score: {ensemble_total:.6f}")
print(f"Baseline total score: {baseline_total:.6f}")
print(f"Improvement: {baseline_total - ensemble_total:.6f}")

In [ ]:
# Save ensemble submission
ensemble_df = pd.DataFrame(ensemble_rows)
ensemble_df.to_csv('/home/submission/submission.csv', index=False)
print(f"Saved ensemble with {len(ensemble_df)} rows")

# Verify
df_verify = pd.read_csv('/home/submission/submission.csv')
verify_score = compute_total_score(df_verify)
print(f"Verified ensemble score: {verify_score:.6f}")

In [ ]:
# Summary
print("="*60)
print("EXPERIMENT 003 SUMMARY: Lattice Construction")
print("="*60)
print(f"Lattice-only score: {lattice_total:.6f}")
print(f"Baseline score: {baseline_total:.6f}")
print(f"Ensemble score: {verify_score:.6f}")
print(f"Improvement over baseline: {baseline_total - verify_score:.6f}")
print("="*60)

In [ ]:
# Model wrapper for submission
class LatticeConstruction:
    def __init__(self, data='single'):
        self.data = data
        
    def load_best(self):
        return pd.read_csv('/home/submission/submission.csv')
    
    def save_submission(self, path):
        df = self.load_best()
        df.to_csv(path, index=False)
        return df

model = LatticeConstruction(data='single')
print("Model wrapper defined")